**<파일 경로>**
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/weather/train

In [1]:
filepath = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/weather/train


In [2]:
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm

# 2018~2019년 기상 데이터 결합

In [3]:
file_1 = pd.read_csv('{}/OBS_AWS_TIM_2018_01~06.csv'.format(filepath), encoding = 'cp949')
file_2 = pd.read_csv('{}/OBS_AWS_TIM_2018_07~12.csv'.format(filepath), encoding = 'cp949')
file_3 = pd.read_csv('{}/OBS_AWS_TIM_2019_01~06.csv'.format(filepath), encoding = 'cp949')
file_4 = pd.read_csv('{}/OBS_AWS_TIM_2019_07~12.csv'.format(filepath), encoding = 'cp949')

In [4]:
concat_list = [file_1, file_2, file_3, file_4]

weather_total = pd.concat(concat_list, axis = 0, ignore_index = True)
weather_total

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),현지기압(hPa),해면기압(hPa),습도(%)
0,116,관악(레),2018-01-01 00:00,-6.0,351.1,6.1,0.0,nan,nan,55.1
1,116,관악(레),2018-01-01 01:00,-6.2,4.3,5.7,0.0,nan,nan,56.8
2,116,관악(레),2018-01-01 02:00,-6.3,357.5,4.9,0.0,nan,nan,53.1
3,116,관악(레),2018-01-01 03:00,-6.2,356.2,4.5,0.0,nan,nan,49.2
4,116,관악(레),2018-01-01 04:00,-6.4,348.0,5.6,0.0,nan,nan,48.7
...,...,...,...,...,...,...,...,...,...,...
496950,889,현충원,2019-12-31 19:00,-5.5,256.0,1.1,0.0,1033.4,1035.5,40.0
496951,889,현충원,2019-12-31 20:00,-5.7,267.2,0.9,0.0,1033.4,1035.5,44.3
496952,889,현충원,2019-12-31 21:00,-5.5,311.1,0.5,0.0,1033.1,1035.2,46.6
496953,889,현충원,2019-12-31 22:00,-8.7,0.0,0.0,0.0,1032.6,1034.8,64.5


# 필요한 변수들만 추출

In [5]:
weather_total = weather_total[['일시', '지점명', '기온(°C)', '풍속(m/s)', '강수량(mm)', '습도(%)']]
weather_total.head()

,일시,지점명,기온(°C),풍속(m/s),강수량(mm),습도(%)
0,2018-01-01 00:00,관악(레),-6.0,6.1,0.0,55.1
1,2018-01-01 01:00,관악(레),-6.2,5.7,0.0,56.8
2,2018-01-01 02:00,관악(레),-6.3,4.9,0.0,53.1
3,2018-01-01 03:00,관악(레),-6.2,4.5,0.0,49.2
4,2018-01-01 04:00,관악(레),-6.4,5.6,0.0,48.7


# 변수명 변경
- 한글로 되어 있는 변수명을 모두 영어로 변경해주겠다.
    - 일시 = date
    - 지점명 = obs_name
    - 기온(°C) = temp
    - 풍속(m/s) = windspeed
    - 강수량(mm) = rain
    - 습도(%) = humidity

In [6]:
col_name = {
    '일시': 'date', '지점명': 'obs_name',
    '기온(°C)': 'temp', '풍속(m/s)': 'windspeed',
    '강수량(mm)': 'rain', '습도(%)': 'humidity'
}

weather_total.rename(columns = col_name, inplace = True)
weather_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496955 entries, 0 to 496954
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       496955 non-null  object 
 1   obs_name   496955 non-null  object 
 2   temp       496422 non-null  float64
 3   windspeed  495707 non-null  float64
 4   rain       493400 non-null  float64
 5   humidity   304560 non-null  float64
dtypes: float64(4), object(2)
memory usage: 22.7+ MB


C:\Users\Brian\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


# 지점명(obs_name) 변수 처리

## 지점명(obs_name) 변수 변경 및 삭제
- 기상 관측 지점명(obs_name) 변수에 대해서 다음과 같은 처리를 해주도록 하겠다.

**<기상 데이터 관측 지점명(obs_name) 변경 및 삭제>**
1. 변경 지점명(obs_name) 리스트
    - 기상청
        - 해당 관측 지점은 주소가 "동작구"이므로, "동작"이라고 변경해주도록 하겠다.
        - 즉, "기상청 >> 동작"
    - 한강
        - 해당 관측 지점은 주소가 "영등포구"이므로, "영등포"라고 변경해주도록 하겠다.
        - 즉, "한강 >> 영등포"
    - 현충원
        - 해당 관측 지점은 주소가 "동작구"이므로, "동작"이라고 변경해주도록 하겠다.
        - 즉, "현충원 >> 동작"
    - 북악산
        - 해당 관측 지점은 주소가 "종로구"이므로, "종로"라고 변경해주도록 하겠다.
        - 즉, "북악산 >> 종로"
2. 삭제 지점명(obs_name) 리스트
    - 관악(레)
        - 해당 관측 지점은 주소가 "경기도"이기 때문에 삭제하도록 하겠다.
    - 남현
        - 해당 관측 지점은 주소가 "전라남도"이기 때문에 삭제하도록 하겠다.

In [7]:
# 지점명 삭제
drop_idx = weather_total[(weather_total['obs_name'] == '관악(레)') | (weather_total['obs_name'] == '남현')].index
weather_total2 = weather_total.drop(drop_idx)

In [8]:
# 지점명 변경
def process_obs_name(x):
    if x == '기상청':
        return '동작'
    elif x == '한강':
        return '영등포'
    elif x == '현충원':
        return '동작'
    elif x == '북악산':
        return '종로'
    else:
        return x

weather_total2['obs_name'] = weather_total2['obs_name'].apply(process_obs_name)

In [9]:
# 서울특별시 자치구 개수 확인
len(set(weather_total2.obs_name))

26

- 작업을 수행해 준 뒤, 자치구의 개수가 25개가 맞는지 확인해보니 26개가 나온다.
    - 1개가 중복되어 나왔다는 말이므로, 어떤 자치구가 중복되었는지 확인해보겠다.

In [10]:
weather_total2.obs_name.value_counts()

동작      34998
성동      17520
동대문     17519
노원      17512
관악      17511
강동      17509
서초      17508
성북      17504
양천      17498
영등포     17498
강북      17497
구로      17496
도봉      17495
서대문     17491
광진      17479
송파      17471
강서      17466
영등포     17434
은평      17424
금천      17398
중구      17394
마포      17375
중랑      17374
용산      17355
강남      17224
종로       8005
Name: obs_name, dtype: int64

- 위 결과를 통해, "영등포"가 중복된 것을 확인하였다.
- 자세히 확인해보니, "영등포" 뒤에 띄어쓰기가 들어간 데이터들이 존재한다.
    - 따라서 이러한 데이터들에 대하여 모두 뒤에 붙은 띄어쓰기를 제거해주도록 하겠다.
    - ex) "영등포 " >> "영등포"

In [11]:
weather_total2[weather_total2['obs_name'] == '영등포']

,date,obs_name,temp,windspeed,rain,humidity
82332,2018-01-01 00:00,영등포,-1.2,2.1,nan,48.5
82333,2018-01-01 01:00,영등포,-1.4,0.7,0.0,45.4
82334,2018-01-01 02:00,영등포,-1.5,1.6,0.0,41.4
82335,2018-01-01 03:00,영등포,-2.0,1.1,0.0,51.0
82336,2018-01-01 04:00,영등포,-2.0,1.1,0.0,52.9
...,...,...,...,...,...,...
461762,2019-12-31 19:00,영등포,-5.0,2.9,0.0,47.2
461763,2019-12-31 20:00,영등포,-5.4,2.8,0.0,42.8
461764,2019-12-31 21:00,영등포,-5.3,2.0,0.0,45.3
461765,2019-12-31 22:00,영등포,-5.8,0.2,0.0,43.8


In [12]:
weather_total2[weather_total2['obs_name'] == '영등포 ']

,date,obs_name,temp,windspeed,rain,humidity
117046,2018-01-01 00:00,영등포,-0.4,1.2,0.0,nan
117047,2018-01-01 01:00,영등포,-0.9,0.3,0.0,nan
117048,2018-01-01 02:00,영등포,-0.8,0.4,0.0,nan
117049,2018-01-01 03:00,영등포,-0.9,0.5,0.0,nan
117050,2018-01-01 04:00,영등포,-1.3,0.3,0.0,nan
...,...,...,...,...,...,...
492542,2019-12-31 19:00,영등포,-5.3,1.8,0.0,nan
492543,2019-12-31 20:00,영등포,-5.3,1.3,0.0,nan
492544,2019-12-31 21:00,영등포,-5.4,0.9,0.0,nan
492545,2019-12-31 22:00,영등포,-5.7,0.3,0.0,nan


In [13]:
def edit_obs_name(x):
    if x == '영등포 ':
        return '영등포'
    else:
        return x
    
weather_total2['obs_name'] = weather_total2['obs_name'].apply(edit_obs_name)
weather_total2.obs_name.value_counts()

동작     34998
영등포    34932
성동     17520
동대문    17519
노원     17512
관악     17511
강동     17509
서초     17508
성북     17504
양천     17498
강북     17497
구로     17496
도봉     17495
서대문    17491
광진     17479
송파     17471
강서     17466
은평     17424
금천     17398
중구     17394
마포     17375
중랑     17374
용산     17355
강남     17224
종로      8005
Name: obs_name, dtype: int64

In [14]:
# 서울특별시 자치구 개수 재확인
len(set(weather_total2.obs_name))

25

### 중복된 값 제거
- 지점명(obs_name)이 "동작"과 영등포"인 경우, 중복된 값들의 존재로 인해 값이 높게 나온 것을 확인할 수 있다.
- 따라서 중복된 값들을 모두 제거해주도록 하겠다.
    - ```drop_duplicates()``` 함수를 사용해서 "date"와 "obs_name" 변수를 기준으로 중복된 값들을 제거해주겠다.
    - 이때 ```keep = 'first'``` 옵션을 통해 첫 번째 값만 남겨주겠다.

In [15]:
weather_total2.drop_duplicates(['date', 'obs_name'], keep = 'first', inplace = True)
weather_total2.obs_name.value_counts()

성동     17520
영등포    17520
동작     17520
동대문    17519
노원     17512
관악     17511
강동     17509
서초     17508
성북     17504
양천     17498
강북     17497
구로     17496
도봉     17495
서대문    17491
광진     17479
송파     17471
강서     17466
은평     17424
금천     17398
중구     17394
마포     17375
중랑     17374
용산     17355
강남     17224
종로      8005
Name: obs_name, dtype: int64

## 지점명(obs_name)별 모든 일자(date) 존재 여부 확인
- 각 지점명(obs_name)별로 모든 일자(date)가 존재하는지 확인해보고, 그렇지 않다면 존재하지 않는 부분들에 대한 처리를 해주도록 하겠다.

In [16]:
# 각 지점명별로 (중복된 값을 제외하고) 총 17520개의 일자가 존재해야 함
len(set(weather_total2['date'])) # 730(날짜) * 24(시간) = 17520개

17520

- 위에서 ```value_counts()``` 함수를 통해, 지점명(obs_name)이 "동작, 성동, 영등포"인 경우를 제외한 나머지 지점들이 모든 일자(date)가 존재하지 않음을 확인할 수 있었다.
    - 즉, 지점명별로 (중복된 값을 제외하고) 총 17520개의 일자가 존재하지 않는다.
- 따라서 이에 대한 처리를 수행해주겠다.

### 각 지점명별로 비어있는 일자(date)를 확인 후, 채워해주는 함수 생성
- 각 지점명별로 비어있는 일자를 찾아내서, 근접한 지점의 값들로 대체해주는 작업을 수행해주도록 하겠다.
    - ex) "강남" 지점의 특정 일자들이 비어있는 경우, 해당 일자의 "송파" 지점 값들로 대체
- 또한 반복 작업을 최소화하기 위해 하나의 함수를 만들어서 사용하겠다.

In [17]:
# 중복 값이 존재하지 않는 일자(date)로 이루어진 DataFrame 생성
date_nodup = pd.DataFrame(weather_total2[weather_total2['obs_name'] == '성동']['date']).reset_index(drop = True)
date_nodup

,date
0,2018-01-01 00:00
1,2018-01-01 01:00
2,2018-01-01 02:00
3,2018-01-01 03:00
4,2018-01-01 04:00
...,...
17515,2019-12-31 19:00
17516,2019-12-31 20:00
17517,2019-12-31 21:00
17518,2019-12-31 22:00


In [18]:
def fill_null_date(null_obs, replace_obs):
    # 지점명이 null_obs로 이루어진 DataFrame 생성
    df_null_obs = weather_total2[weather_total2['obs_name'] == null_obs].reset_index(drop = True)

    # date_nodup과 df_null_obs을 left join
    df_null_check = pd.merge(date_nodup, df_null_obs, how = 'left', on = 'date')
    # 이를 통해 비어있는 일자(date) 확인
    df_null_date = df_null_check[df_null_check['obs_name'].isnull()][['date']]

    # null_obs 지점의 비어있는 일자(date)에 대해서 replace_obs 지점의 관측값들로 대체
    df_replace_obs = weather_total2[weather_total2['obs_name'] == replace_obs].reset_index(drop = True)
    df_null_date2 = pd.merge(df_null_date, df_replace_obs, how = 'left', on = 'date')

    # replace_obs라고 되어 있는 지점명을 null_obs으로 변경
    def change_obs_name(x):
        if x == replace_obs:
            return null_obs

    df_null_date2['obs_name'] = df_null_date2['obs_name'].apply(change_obs_name)

    # 지점명이 null_obs로 이루어진 DataFrame에 비어있는 일자(date)가 채워진 DataFrame을 결합
    df_null_obs2 = pd.concat([df_null_obs, df_null_date2], axis = 0)
    df_null_obs2.sort_values(by = 'date', inplace = True)
    
    return df_null_obs2.reset_index(drop = True)

In [19]:
weather_total2.obs_name.value_counts()

성동     17520
영등포    17520
동작     17520
동대문    17519
노원     17512
관악     17511
강동     17509
서초     17508
성북     17504
양천     17498
강북     17497
구로     17496
도봉     17495
서대문    17491
광진     17479
송파     17471
강서     17466
은평     17424
금천     17398
중구     17394
마포     17375
중랑     17374
용산     17355
강남     17224
종로      8005
Name: obs_name, dtype: int64

In [20]:
# "송파" 지점에 비어있는 일자(date) 처리
obs_송파 = fill_null_date('송파', '강남')
# "강서" 지점에 비어있는 일자(date) 처리
obs_강서 = fill_null_date('강서', '양천')
# "은평" 지점에 비어있는 일자(date) 처리
obs_은평 = fill_null_date('은평', '성북')
# "금천" 지점에 비어있는 일자(date) 처리
obs_금천 = fill_null_date('금천', '구로')
# "중구" 지점에 비어있는 일자(date) 처리
obs_중구 = fill_null_date('중구', '성동')
# "마포" 지점에 비어있는 일자(date) 처리
obs_마포 = fill_null_date('마포', '영등포')
# "중랑" 지점에 비어있는 일자(date) 처리
obs_중랑 = fill_null_date('중랑', '동대문')
# "용산" 지점에 비어있는 일자(date) 처리
obs_용산 = fill_null_date('용산', '중구')
# "강남" 지점에 비어있는 일자(date) 처리
obs_강남 = fill_null_date('강남', '송파')
# "동대문" 지점에 비어있는 일자(date) 처리
obs_동대문 = fill_null_date('동대문', '성동')
# "노원" 지점에 비어있는 일자(date) 처리
obs_노원 = fill_null_date('노원', '도봉')
# "관악" 지점에 비어있는 일자(date) 처리
obs_관악 = fill_null_date('관악', '동작')
# "강동" 지점에 비어있는 일자(date) 처리
obs_강동 = fill_null_date('강동', '송파')
# "서초" 지점에 비어있는 일자(date) 처리
obs_서초 = fill_null_date('서초', '강남')
# "성북" 지점에 비어있는 일자(date) 처리
obs_성북 = fill_null_date('성북', '동대문')
# "양천" 지점에 비어있는 일자(date) 처리
obs_양천 = fill_null_date('양천', '구로')
# "강북" 지점에 비어있는 일자(date) 처리
obs_강북 = fill_null_date('강북', '도봉')
# "구로" 지점에 비어있는 일자(date) 처리
obs_구로 = fill_null_date('구로', '양천')
# "도봉" 지점에 비어있는 일자(date) 처리
obs_도봉 = fill_null_date('도봉', '노원')
# "서대문" 지점에 비어있는 일자(date) 처리
obs_서대문 = fill_null_date('서대문', '종로')
# "광진" 지점에 비어있는 일자(date) 처리
obs_광진 = fill_null_date('광진', '성동')
# "종로" 지점에 비어있는 일자(date) 처리
obs_종로 = fill_null_date('종로', '서대문')

In [21]:
check_null_count = [obs_송파.obs_name.isnull().sum(), obs_강서.obs_name.isnull().sum(), obs_은평.obs_name.isnull().sum(),
                    obs_금천.obs_name.isnull().sum(), obs_중구.obs_name.isnull().sum(), obs_마포.obs_name.isnull().sum(),
                    obs_중랑.obs_name.isnull().sum(), obs_용산.obs_name.isnull().sum(), obs_강남.obs_name.isnull().sum(),
                    obs_동대문.obs_name.isnull().sum(), obs_노원.obs_name.isnull().sum(), obs_관악.obs_name.isnull().sum(),
                    obs_강동.obs_name.isnull().sum(), obs_서초.obs_name.isnull().sum(), obs_성북.obs_name.isnull().sum(),
                    obs_양천.obs_name.isnull().sum(), obs_강북.obs_name.isnull().sum(), obs_구로.obs_name.isnull().sum(),
                    obs_도봉.obs_name.isnull().sum(), obs_서대문.obs_name.isnull().sum(), obs_광진.obs_name.isnull().sum(),
                    obs_종로.obs_name.isnull().sum()]
check_null_count

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]

- 각 지점별로 비어있는 일자(date)와 해당 변수들을 인접한 지점의 값들로 채워넣은 뒤, 각 지점별로 비어있는 일자(date)가 존재하는지 재확인해보았다.
- 그 결과, 위와 같이 "서대문"과 "종로" 지점에는 여전히 각각 1개의 결측 일자(date)가 존재함을 확인할 수 있다.
    - 즉, "서대문"과 "종로" 지점의 경우에는 인접한 지점의 값으로 대체를 하는 과정에서, 각각 1개의 일자(date)에 대해서 대체를 하지 못 했음을 의미한다.

- 따라서 "서대문"과 "종로" 지점에서 대체되지 못한 일자(date)를 찾아내서, 앞/뒤 값의 평균 값으로 대체해주도록 하겠다.
    - ex) "2019-01-01 09:00"에 해당하는 자료가 NaN인 경우, 모든 변수들을 "2019-01-01 08:00"과 "2019-01-01 10:00" 자료의 평균 값으로 대체

In [22]:
# "서대문" 지점에 존재하는 결측 일자(date) 확인
obs_서대문[obs_서대문['obs_name'].isnull()] # 14234번째 index

,date,obs_name,temp,windspeed,rain,humidity
14234,2019-08-17 02:00,None,nan,nan,nan,nan


In [23]:
# "종로" 지점에 존재하는 결측 일자(date) 확인
obs_종로[obs_종로['obs_name'].isnull()] # 14234번째 index

,date,obs_name,temp,windspeed,rain,humidity
14234,2019-08-17 02:00,None,nan,nan,nan,nan


In [24]:
# 1. "서대문" 지점 >> 결측 일자(date) 기준 "서대문" 지점의 앞/뒤 평균 값으로 대체
# 14234번째 index에 결측 일자(date)가 존재하므로, 14233번과 14235번째 index의 자료 값들의 평균을 구해서 대체
val_list = ['temp', 'windspeed', 'rain', 'humidity'] # 평균 값으로 대체해 줄 변수들
for val in val_list:
    mean_val = round((obs_서대문.loc[14233, val] + obs_서대문.loc[14235, val]) / 2, 1)
    obs_서대문.loc[14234, val] = mean_val

obs_서대문['obs_name'].fillna('서대문', inplace = True)

In [25]:
# 남아있던 1개의 결측 일자(date)가 사라졌는지 확인
obs_서대문.obs_name.isnull().sum()

0

In [26]:
# 2. "종로" 지점 >> 결측 일자(date) 기준 "종로" 지점의 앞/뒤 평균 값으로 대체
# 14234번째 index에 결측 일자(date)가 존재하므로, 14233번과 14235번째 index의 자료 값들의 평균을 구해서 대체
val_list = ['temp', 'windspeed', 'rain', 'humidity'] # 평균 값으로 대체해 줄 변수들
for val in val_list:
    mean_val = round((obs_종로.loc[14233, val] + obs_종로.loc[14235, val]) / 2, 1)
    obs_종로.loc[14234, val] = mean_val

obs_종로['obs_name'].fillna('종로', inplace = True)

In [27]:
# 남아있던 1개의 결측 일자(date)가 사라졌는지 확인
obs_종로.obs_name.isnull().sum()

0

### 모든 DataFrame을 하나로 결합
- 비어있는 일자(date)와 그에 해당되는 변수 값들을 채워넣은 각 지점명별 DataFrame들과, 일자(date)가 비어있지 않았던 "성동, 동작, 영등포" 지점의 DataFrame들을 결합해서 하나의 DataFrame으로 만들어주겠다.

In [28]:
obs_성동 = weather_total2[weather_total2['obs_name'] == '성동'].reset_index(drop = True)
obs_동작 = weather_total2[weather_total2['obs_name'] == '동작'].reset_index(drop = True)
obs_영등포 = weather_total2[weather_total2['obs_name'] == '영등포'].reset_index(drop = True)

In [29]:
# 확인용
(obs_성동.obs_name.isnull().sum(), obs_동작.obs_name.isnull().sum(), obs_영등포.obs_name.isnull().sum())

(0, 0, 0)

In [30]:
concat_list = [obs_성동, obs_동작, obs_영등포, obs_송파, obs_강서, obs_은평, obs_금천, obs_중구,
               obs_마포, obs_중랑, obs_용산, obs_강남, obs_동대문, obs_노원, obs_관악, obs_강동,
               obs_서초, obs_성북, obs_양천, obs_강북, obs_구로, obs_도봉, obs_서대문, obs_광진, obs_종로]
obs_total = pd.concat(concat_list, axis = 0, ignore_index = True)
obs_total

,date,obs_name,temp,windspeed,rain,humidity
0,2018-01-01 00:00,성동,-0.1,1.0,0.0,41.2
1,2018-01-01 01:00,성동,-0.2,1.7,0.0,39.8
2,2018-01-01 02:00,성동,-0.4,1.5,0.0,41.4
3,2018-01-01 03:00,성동,-0.5,2.0,0.0,42.8
4,2018-01-01 04:00,성동,-0.8,2.1,0.0,45.4
...,...,...,...,...,...,...
437995,2019-12-31 19:00,종로,-6.9,1.7,0.0,nan
437996,2019-12-31 20:00,종로,-7.0,1.3,0.0,nan
437997,2019-12-31 21:00,종로,-8.1,1.0,0.0,nan
437998,2019-12-31 22:00,종로,-8.6,0.8,0.0,nan


In [31]:
obs_total.obs_name.isnull().sum()

0

In [32]:
obs_total.obs_name.value_counts()

금천     17520
서초     17520
관악     17520
강북     17520
강동     17520
성북     17520
도봉     17520
강서     17520
은평     17520
용산     17520
동대문    17520
영등포    17520
강남     17520
동작     17520
송파     17520
마포     17520
중구     17520
광진     17520
성동     17520
중랑     17520
구로     17520
양천     17520
종로     17520
노원     17520
서대문    17520
Name: obs_name, dtype: int64

# 일시(date) 변수 처리
- date 변수의 타입을 datetime으로 변경하고, "시간대"를 의미하는 hour 변수를 만들어주도록 하겠다.
- 그 다음, date 변수의 형태를 다음과 같이 변경해주겠다.
    - ex) "2018-01-01 05:00:00" >> "2018-01-01"

In [33]:
# 최종 데이터 셋을 구성하기 위해 복사본 사용
weather_final = obs_total.copy()

In [34]:
# datetime 타입으로 변경
weather_final['date'] = pd.to_datetime(weather_final['date'])

# hour 변수 생성
weather_final['hour'] = weather_final['date'].dt.hour
# date 변수 생성(기존의 date 변수에 덮어쓰기)
weather_final['date'] = weather_final['date'].dt.date

In [35]:
weather_final

,date,obs_name,temp,windspeed,rain,humidity,hour
0,2018-01-01,성동,-0.1,1.0,0.0,41.2,0
1,2018-01-01,성동,-0.2,1.7,0.0,39.8,1
2,2018-01-01,성동,-0.4,1.5,0.0,41.4,2
3,2018-01-01,성동,-0.5,2.0,0.0,42.8,3
4,2018-01-01,성동,-0.8,2.1,0.0,45.4,4
...,...,...,...,...,...,...,...
437995,2019-12-31,종로,-6.9,1.7,0.0,nan,19
437996,2019-12-31,종로,-7.0,1.3,0.0,nan,20
437997,2019-12-31,종로,-8.1,1.0,0.0,nan,21
437998,2019-12-31,종로,-8.6,0.8,0.0,nan,22


In [36]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438000 entries, 0 to 437999
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       438000 non-null  object 
 1   obs_name   438000 non-null  object 
 2   temp       437499 non-null  float64
 3   windspeed  436992 non-null  float64
 4   rain       434810 non-null  float64
 5   humidity   253023 non-null  float64
 6   hour       438000 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 23.4+ MB


# 지점명(obs_name) 변수 값 형태 및 변수명 변경
- 추후에 최종 데이터 셋을 만들 때 조인키로 사용하기 위해, "지점명" 변수 값의 형태를 다음과 같이 바꿔주겠다,
    - ex) "강남" >> "강남구"
    - 단, "중구"의 경우 수정 작업이 필요 없다. (이미 자치구명 형태로 입력되어 있기 때문)
        - 따라서 먼저 모든 "지점명" 변수 값의 형태를 변경해 준 뒤, "중구구"라고 변경되었을 "중구" 지점에 대해서만 "구" 문자열 하나를 지워주도록 하겠다.
        - ex) "중구구" >> "중구"
- 그리고 "지점명(obs_name)"이 의미하는 바가 자치구와 동일하므로, 변수명을 "gu"라고 변경해주겠다.

In [37]:
# 변수 값의 형태 변경
weather_final['obs_name'] = weather_final['obs_name'] + '구'

# "중구구" >> "중구" 로 수정
def jung_gu_name(x):
    if x == '중구구':
        return '중구'
    else:
        return x
weather_final['obs_name'] = weather_final['obs_name'].apply(jung_gu_name)

# 변수명 변경
weather_final.rename(columns = {'obs_name': 'gu'}, inplace = True)
weather_final

,date,gu,temp,windspeed,rain,humidity,hour
0,2018-01-01,성동구,-0.1,1.0,0.0,41.2,0
1,2018-01-01,성동구,-0.2,1.7,0.0,39.8,1
2,2018-01-01,성동구,-0.4,1.5,0.0,41.4,2
3,2018-01-01,성동구,-0.5,2.0,0.0,42.8,3
4,2018-01-01,성동구,-0.8,2.1,0.0,45.4,4
...,...,...,...,...,...,...,...
437995,2019-12-31,종로구,-6.9,1.7,0.0,nan,19
437996,2019-12-31,종로구,-7.0,1.3,0.0,nan,20
437997,2019-12-31,종로구,-8.1,1.0,0.0,nan,21
437998,2019-12-31,종로구,-8.6,0.8,0.0,nan,22


In [38]:
weather_final.gu.value_counts()

금천구     17520
중구      17520
영등포구    17520
광진구     17520
성북구     17520
용산구     17520
강북구     17520
동대문구    17520
동작구     17520
구로구     17520
은평구     17520
서대문구    17520
노원구     17520
양천구     17520
관악구     17520
송파구     17520
강서구     17520
강동구     17520
마포구     17520
성동구     17520
서초구     17520
강남구     17520
중랑구     17520
종로구     17520
도봉구     17520
Name: gu, dtype: int64

# 조인키 생성
- 지하철 데이터 셋을 기준으로 인구 데이터 셋과 기상 데이터 셋을 결합시켜 줄 것이므로, 이를 위한 조인키가 필요하다.
- 따라서 지하철 데이터 셋과 기상 데이터 셋을 결합하기 위한 조인키 1개를 만들어주도록 하겠다.
    - 지하철 데이터 셋과 기상 데이터 셋을 결합하기 위한 조인키(weather_join_key)
        - ex) "2018-01-01_중구"

In [39]:
# datetime to string
weather_final['date'] = weather_final['date'].astype(str)

# 지하철 데이터와 결합할 때 사용할 조인키
weather_final['weather_join_key'] = weather_final['date'] + '_' + weather_final['gu']
weather_final['weather_join_key']

0         2018-01-01_성동구
1         2018-01-01_성동구
2         2018-01-01_성동구
3         2018-01-01_성동구
4         2018-01-01_성동구
               ...      
437995    2019-12-31_종로구
437996    2019-12-31_종로구
437997    2019-12-31_종로구
437998    2019-12-31_종로구
437999    2019-12-31_종로구
Name: weather_join_key, Length: 438000, dtype: object

# 결측치 처리
- humidity 변수의 경우, 결측치가 너무 많기 때문에 분석 대상에서 제외시키도록 하겠다.
    - 즉, humidity 변수는 그냥 제거해주겠다.
- temp, windspeed, rain 변수의 경우에는 결측치들을 앞/뒤 값의 평균 값으로 대체해주겠다.
    - 만약 앞/뒤 값 중, 결측치가 하나라도 존재해서 대체가 안 되는 경우에는 전체 평균 값으로 대체해주도록 하겠다.

In [40]:
weather_final.isnull().sum()

date                     0
gu                       0
temp                   501
windspeed             1008
rain                  3190
humidity            184977
hour                     0
weather_join_key         0
dtype: int64

In [41]:
# humidity 변수 제거
weather_final.drop(['humidity'], axis = 1, inplace = True)

In [42]:
# temp 변수에서 결측치들이 존재하는 index 추출
temp_null_idx = weather_final[weather_final['temp'].isnull()].index
# windspeed 변수에서 결측치들이 존재하는 index 추출
windspeed_null_idx = weather_final[weather_final['windspeed'].isnull()].index
# rain 변수에서 결측치들이 존재하는 index 추출
rain_null_idx = weather_final[weather_final['rain'].isnull()].index

In [43]:
# temp 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(temp_null_idx):
    temp_fillna = round(weather_final.loc[idx - 1, 'temp'] + weather_final.loc[idx + 1, 'temp'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'temp'] = temp_fillna

# windspeed 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(windspeed_null_idx):
    windspeed_fillna = round(weather_final.loc[idx - 1, 'windspeed'] + weather_final.loc[idx + 1, 'windspeed'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'windspeed'] = windspeed_fillna
    
# rain 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(rain_null_idx):
    rain_fillna = round(weather_final.loc[idx - 1, 'rain'] + weather_final.loc[idx + 1, 'rain'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'rain'] = rain_fillna

In [44]:
weather_final.isnull().sum()

date                   0
gu                     0
temp                 295
windspeed            525
rain                3150
hour                   0
weather_join_key       0
dtype: int64

- 위 결과를 보면, 여전히 결측치가 존재한다.
    - 즉, 앞/뒤 값 중 하나라도 결측인 값이 존재했다는 의미이다.
- 따라서 이 경우들에 대해서는 전체 평균 값으로 대체해주도록 하겠다.

In [45]:
# temp 변수 전체 평균 값
temp_total_mean = round(np.mean(weather_final['temp']), 1)
# windspeed 변수 전체 평균 값
windspeed_total_mean = round(np.mean(weather_final['windspeed']), 1)
# rain 변수 전체 평균 값
rain_total_mean = round(np.mean(weather_final['rain']), 1)

weather_final['temp'].fillna(temp_total_mean, inplace = True)
weather_final['windspeed'].fillna(windspeed_total_mean, inplace = True)
weather_final['rain'].fillna(rain_total_mean, inplace = True)

In [46]:
weather_final.isnull().sum()

date                0
gu                  0
temp                0
windspeed           0
rain                0
hour                0
weather_join_key    0
dtype: int64

# 불필요한 변수들 제거
- gu 변수는 분석에 사용하지 않을 것이므로 제거해주겠다.

In [47]:
weather_final.columns

Index(['date', 'gu', 'temp', 'windspeed', 'rain', 'hour', 'weather_join_key'], dtype='object')

In [48]:
weather_final.drop(['gu'], axis = 1, inplace = True)

In [49]:
weather_final

,date,temp,windspeed,rain,hour,weather_join_key
0,2018-01-01,-0.1,1.0,0.0,0,2018-01-01_성동구
1,2018-01-01,-0.2,1.7,0.0,1,2018-01-01_성동구
2,2018-01-01,-0.4,1.5,0.0,2,2018-01-01_성동구
3,2018-01-01,-0.5,2.0,0.0,3,2018-01-01_성동구
4,2018-01-01,-0.8,2.1,0.0,4,2018-01-01_성동구
...,...,...,...,...,...,...
437995,2019-12-31,-6.9,1.7,0.0,19,2019-12-31_종로구
437996,2019-12-31,-7.0,1.3,0.0,20,2019-12-31_종로구
437997,2019-12-31,-8.1,1.0,0.0,21,2019-12-31_종로구
437998,2019-12-31,-8.6,0.8,0.0,22,2019-12-31_종로구


In [50]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438000 entries, 0 to 437999
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              438000 non-null  object 
 1   temp              438000 non-null  float64
 2   windspeed         438000 non-null  float64
 3   rain              438000 non-null  float64
 4   hour              438000 non-null  int64  
 5   weather_join_key  438000 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 20.1+ MB


# 최종 생성된 데이터 셋을 csv 파일로 내보내기

In [51]:
new_filepath = 'C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/train'

In [52]:
weather_final.to_csv('{}/weather_final.csv'.format(new_filepath), index = False)